# 91_fig1_ergodic_distributions

Figure 1-style ergodic distributions by regime (uses `sim_paths.npz` from commitment run).

In [ ]:
import os, sys, json, numpy as np, torch, matplotlib.pyplot as plt
sys.path.append("..")

from src.config import ModelParams
from src.io_utils import load_json, load_npz, load_torch, load_selected_run, find_latest_run_dir
from src.deqn import PolicyNetwork

ART = os.path.join("..","artifacts","runs")

def get_run(policy: str) -> str:
    rd = load_selected_run(ART, policy)
    if rd is None:
        rd = find_latest_run_dir(ART, policy)
    if rd is None:
        raise RuntimeError(f"No runs found for policy={policy} under {ART}")
    return rd

def _parse_dtype(s: str):
    if s is None:
        return torch.float32
    if isinstance(s, torch.dtype):
        return s
    s = str(s)
    if "float64" in s:
        return torch.float64
    if "float32" in s:
        return torch.float32
    if "bfloat16" in s:
        return torch.bfloat16
    return torch.float32

def load_params_from_run(run_dir: str, *, device="cpu"):
    cfg = load_json(os.path.join(run_dir, "config.json"))
    p = cfg.get("params", {})
    dtype = _parse_dtype(p.get("dtype"))
    dev = device if device is not None else p.get("device","cpu")
    keep = {k:v for k,v in p.items() if k in ModelParams.__dataclass_fields__}
    keep["device"] = dev
    keep["dtype"] = dtype
    return ModelParams(**keep).to_torch()

def load_net_from_run(run_dir: str, d_in: int, d_out: int):
    cfg = load_json(os.path.join(run_dir, "config.json"))
    tc = cfg.get("train_cfg", {})
    hidden = tuple(tc.get("hidden_layers", (512,512)))
    activation = tc.get("activation", "selu")
    net = PolicyNetwork(d_in, d_out, hidden=hidden, activation=activation)
    state = load_torch(os.path.join(run_dir, "weights.pt"), map_location="cpu")
    # state is usually a plain state_dict
    if isinstance(state, dict) and "state_dict" in state:
        state = state["state_dict"]
    net.load_state_dict(state)
    net.eval()
    return net

# --- paper reporting helpers ---
ann = lambda x: 400.0*x  # annualized percent (quarterly -> annual)


In [ ]:
run = get_run("commitment")
sim_path = os.path.join(run, "sim_paths.npz")
if not os.path.exists(sim_path):
    raise FileNotFoundError(f"Missing sim_paths.npz in {run}. Re-run training notebook with simulation enabled.")
sim = load_npz(sim_path)

s = sim["s"].reshape(-1).astype(np.int64)

def hist_by_regime(arr, title, bins=60, scale=1.0):
    x = arr.reshape(-1) * float(scale)
    plt.figure()
    plt.hist(x[s==0], bins=bins, alpha=0.6, label="normal (s=0)")
    plt.hist(x[s==1], bins=bins, alpha=0.6, label="bad (s=1)")
    plt.title(title)
    plt.legend()
    plt.show()

plot_specs = [
    ("pi", "Figure 1 proxy: Inflation pi distribution (annualized %)", 400.0),
    ("i", "Figure 1 proxy: Nominal rate i distribution (annualized %)", 400.0),
    ("c", "Figure 1 proxy: Consumption c distribution (level)", 1.0),
    ("y", "Figure 1 proxy: Output y distribution (level)", 1.0),
    ("Delta", "Figure 1 proxy: Price dispersion Delta distribution (level)", 1.0),
]
for var, ttl, scale in plot_specs:
    if var in sim:
        hist_by_regime(sim[var], ttl, scale=scale)
